<a href="https://colab.research.google.com/github/Pranjalya/fake-news-twitter/blob/get-label/Fact_Check_News_Labelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Label Extractor

Downloading data.

In [1]:
!curl 'https://storage.googleapis.com/kaggle-data-sets/689642%2F1213945%2Fcompressed%2FAllTweets_Dec2019_June2020.csv.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1591776960&Signature=lOPPi5k%2FlvyJT8uawwMzZsSZ1m%2B2XT2ti%2BMWBQzXcAj5zPbsMepoaGfQ2pRYai3aZEAc9R9R8Y8BtSYKP5fEfwpN1SyOaRtYcU%2BsqnK7xWVqc5cNYl3sBzOiYBZDOA1DnaopD213MF1EjNtdcg3tFzbZ9X%2FReXzEuHShkPtFjfWudnchNRNKkMpLw5twvLxFtyrWt4tMlTdLo7E35AdDedW%2FEoBC1dOGUvGrdg3%2Feh%2F6B0H7WM25i79%2FblRU5Ef7oU%2F5lfMUeNFfEpuxmOdvsq8jvlAxUAS5FXpKvFVvmBPawEBAnkp6p9LYyOg3unAaHLLGQVxQAQdFzTVtUt8o6Q%3D%3D' -H 'authority: storage.googleapis.com' -H 'upgrade-insecure-requests: 1' -H 'dnt: 1' -H 'user-agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.132 Safari/537.36' -H 'sec-fetch-dest: document' -H 'accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9' -H 'sec-fetch-site: cross-site' -H 'sec-fetch-mode: navigate' -H 'sec-fetch-user: ?1' -H 'referer: https://www.kaggle.com/' -H 'accept-language: en-IN,en-GB;q=0.9,en-US;q=0.8,en;q=0.7' --compressed --output fact_checker.zip
!mkdir data
!unzip fact_checker.zip -d data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 10.1M  100 10.1M    0     0  26.7M      0 --:--:-- --:--:-- --:--:-- 26.7M
Archive:  fact_checker.zip
  inflating: data/AllTweets_Dec2019_June2020.csv  


## Study and Analysis

In [2]:
import pandas as pd
import re
import os
import numpy as np

fcn = pd.read_csv('/content/data/AllTweets_Dec2019_June2020.csv')
fcn.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,conversation_id,created_at,date,timezone,place,tweet,hashtags,cashtags,user_id,user_id_str,username,name,day,hour,link,retweet,nlikes,nreplies,nretweets,quote_url,search,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,Region
0,1267487446809313282,1267487446809313282,1591027530000,2020-06-01 16:05:30,UTC,NaN,"Aarti Lalchandani, principal of GVSM Medical C...",['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,1,16,https://twitter.com/AltNews/status/12674874468...,False,811,40,373,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others
1,1267108282164019202,1267108282164019202,1590937130000,2020-05-31 14:58:50,UTC,NaN,Times Now played three videos during a show on...,['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,7,14,https://twitter.com/AltNews/status/12671082821...,False,278,9,92,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others
2,1266732527227744256,1266732527227744256,1590847543000,2020-05-30 14:05:43,UTC,NaN,A video of an incident from Surat has been sha...,['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,6,14,https://twitter.com/AltNews/status/12667325272...,False,148,1,56,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others
3,1266671072671391744,1266671072671391744,1590832891000,2020-05-30 10:01:31,UTC,NaN,A video showing poor condition of Mumbai's KEM...,['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,6,10,https://twitter.com/AltNews/status/12666710726...,False,157,5,64,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others
4,1266653343767597057,1266653343767597057,1590828665000,2020-05-30 08:51:05,UTC,NaN,"Criticising journalists on COVID-19 reporting,...",['#altnewsfactcheck'],[],717020664587227136,717020664587227136,AltNews,Alt News,6,8,https://twitter.com/AltNews/status/12666533437...,False,439,25,196,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,"[{'user_id': '717020664587227136', 'username':...",NaN,NaN,NaN,NaN,Others


In [0]:
group = fcn.groupby('username')
sample = pd.DataFrame(columns = fcn.columns)
for key, value in group:
    sample = pd.concat([sample, value.head(2)])

In [4]:
for x, y in zip(list(sample['username']), list(sample['tweet'])):
    print("{} : {}".format(x, y))

ABCFactCheck : CORRECTION: in an earlier tweet, we paraphrased @zalisteggall as saying there are 60,000 people employed in coal mining. She actually said "in fossil-fuel and coal mining". Here's our fact check on the issue (not including oil and gas numbers):  https://ab.co/2JzVUeM  #qanda
ABCFactCheck : Thanks Zali, you said (according to TVeeder): "there's 242,000 jobs in mining, of which it's only about 60,000 that are in fossil-fuel and coal mining." Apologies for the misquote.

FTR, your number (38,000 in coal mining) accords with one of the data sets in our fact check
AfpFactuel : Conversions massives à l’islam, musulmans immunisés face au virus : dans le déchaînement international de désinformation suscité par le #Covid_19, les théoriciens du complot au Moyen-Orient ont, eux, plongé dans un excès de religiosité  http://u.afp.com/3doz  #AFP
AfpFactuel : Foisonnantes sur les réseaux sociaux du monde entier, les théories complotistes présentant Bill Gates en grand orchestrateur du 